# PII 마스킹 with Local LLM (Gemma3:27b)

## 목적
- CSV 대화 데이터에서 개인정보(PII) 마스킹
- 외부 API(Judge)에 안전하게 전달하기 위한 전처리

## 마스킹 대상
1. **직접 식별정보**: 이름, 이메일, 전화번호, 주소, 주민등록번호
2. **민감정보(건강)**: 질병/진단명, 검사 수치, 검진 결과, 검진 날짜, 복용 약물

## 1. 환경 설정

In [16]:
import os
import re
import pandas as pd
import requests
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gemma3:27b"

print(f"Ollama URL: {OLLAMA_URL}")
print(f"Model: {MODEL_NAME}")

Ollama URL: http://171.0.53.81:11434
Model: gemma3:27b


In [17]:
# Ollama 서버 연결 확인
try:
    response = requests.get(f"{OLLAMA_URL}/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        model_names = [m["name"] for m in models]
        print("Ollama 서버 연결 성공!")
        print(f"사용 가능한 모델: {model_names}")
        if not any(MODEL_NAME in m for m in model_names):
            print(f"\n[경고] {MODEL_NAME} 모델이 없습니다.")
            print(f"다운로드: ollama pull {MODEL_NAME}")
except requests.exceptions.ConnectionError:
    print("Ollama 서버에 연결할 수 없습니다.")
    print("실행 확인: ollama serve")

Ollama 서버 연결 성공!
사용 가능한 모델: ['qwen2.5:72b', 'alibayram/medgemma:27b', 'qwen3:32b', 'qwen3:30b', 'qwen3:14b', 'qwen3:8b', 'embeddinggemma:300m', 'qwen3-coder:30b', 'deepseek-r1:70b', 'deepseek-r1:32b', 'bge-m3:567m', 'gemma3:4b', 'gemma3:27b', 'gemma3:12b', 'gpt-oss:120b', 'gpt-oss:20b']


## 2. LLM 설정

In [18]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOllama(
    model=MODEL_NAME,
    base_url=OLLAMA_URL,
    temperature=0.1,
)

print(f"{MODEL_NAME} 모델 설정 완료!")

gemma3:27b 모델 설정 완료!


## 3. 마스킹 프롬프트 및 함수 정의

In [19]:
MASKING_SYSTEM_PROMPT = """당신은 개인정보 마스킹 전문가입니다.
주어진 텍스트에서 개인정보를 찾아 플레이스홀더로 대체하세요.

## 마스킹 대상 및 플레이스홀더

### 1. 직접 식별정보
- 이름 (사람 이름) -> [NAME]
- 이메일 주소 -> [EMAIL]
- 전화번호 -> [PHONE]
- 주소 -> [ADDRESS]
- 주민등록번호 -> [SSN]

### 2. 민감정보 (건강)
- 질병/진단명 (예: 당뇨, 고혈압, 지방간 등) -> [CONDITION]
- 검사 수치 (예: 115mg/dL, 130/85mmHg 등) -> [HEALTH_VALUE]
- 검진 결과/판정 (예: 정상A, 정상B, 주의 등) -> [RESULT]
- 특정 검진 날짜 (예: 2024년 3월 15일) -> [DATE]
- 복용 약물 -> [MEDICATION]

## 규칙
1. 마스킹 대상만 플레이스홀더로 대체하고, 나머지 텍스트는 그대로 유지하세요.
2. 같은 유형의 정보는 동일한 플레이스홀더를 사용하세요.
3. 나이대(30대), 성별(여성) 같은 일반적 정보는 마스킹하지 마세요.
4. 일반적인 건강 조언, 음식명, 운동명은 마스킹하지 마세요.
5. 마스킹된 텍스트만 출력하세요. 설명이나 주석은 추가하지 마세요.

## 예시

입력: 홍길동님, 2024년 3월 15일 건강검진 결과 공복혈당이 115mg/dL로 고혈압이 경계 수준입니다.
출력: [NAME]님, [DATE] 건강검진 결과 공복혈당이 [HEALTH_VALUE]로 [CONDITION]이 경계 수준입니다.

입력: example123@company.com 계정으로 로그인하셨습니다.
출력: [EMAIL] 계정으로 로그인하셨습니다.
"""

In [20]:
def create_masking_prompt(text: str) -> str:
    """마스킹 요청 프롬프트 생성"""
    return f"""다음 텍스트에서 개인정보를 마스킹하세요.

텍스트:
{text}

마스킹된 텍스트:"""


def mask_text(text: str) -> str:
    """텍스트 마스킹 수행"""
    if not text or pd.isna(text):
        return text
    
    try:
        messages = [
            SystemMessage(content=MASKING_SYSTEM_PROMPT),
            HumanMessage(content=create_masking_prompt(text))
        ]
        response = llm.invoke(messages)
        return response.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return text

## 4. 마스킹 테스트

In [21]:
test_text = "김영수님, 2024년 5월 20일 건강검진 결과 총콜레스테롤이 245mg/dL로 고지혈증 주의가 필요합니다."

print("=" * 50)
print("[원본]")
print(test_text)
print("=" * 50)
print("[마스킹 결과]")
print(mask_text(test_text))
print("=" * 50)

[원본]
김영수님, 2024년 5월 20일 건강검진 결과 총콜레스테롤이 245mg/dL로 고지혈증 주의가 필요합니다.
[마스킹 결과]
[NAME]님, [DATE] 건강검진 결과 총콜레스테롤이 [HEALTH_VALUE]로 [CONDITION] 주의가 필요합니다.


## 5. CSV 데이터 로드

In [ ]:
df = pd.read_csv("MESSAGE_INFO_REAL.csv")

print(f"총 {len(df)}개 메시지")
print(f"세션 수: {df['session_id'].nunique()}개")
df.head()

총 486개 메시지
세션 수: 40개


,#,email,session_id,role,content,created_at
0,1,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,USER,"안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질...",2025-10-01 06:04:52.205165 +00:00
1,2,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"조미진님, 안녕하세요. 2023년 11월 17일 건강검진 결과를 바탕으로 건강 상태...",2025-10-01 06:05:30.045710 +00:00
2,3,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,USER,신장(콩팥) 개선 방법을 알려줘,2025-10-01 06:05:37.811683 +00:00
3,4,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"## 신장기능 관리를 위한 3가지 핵심 방법\n\n조미진님, 단백뇨 관리를 위해 다...",2025-10-01 06:05:51.403698 +00:00
4,5,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,USER,신장기능 식단 관리 방법을 알려줘,2025-10-01 06:06:11.208222 +00:00


## 6. 전체 CSV 마스킹 실행

In [23]:
def mask_dataframe(df: pd.DataFrame, content_column: str = "content") -> pd.DataFrame:
    """DataFrame의 content 컬럼 전체 마스킹"""
    df_masked = df.copy()
    df_masked["email"] = "[EMAIL]"
    
    masked_contents = []
    for idx, row in tqdm(df_masked.iterrows(), total=len(df_masked), desc="마스킹 진행"):
        masked_content = mask_text(row[content_column])
        masked_contents.append(masked_content)
        
        if (idx + 1) % 10 == 0:
            print(f"[{idx + 1}/{len(df_masked)}] 처리 완료")
    
    df_masked[content_column] = masked_contents
    return df_masked

In [24]:
# 테스트: 처음 20개만 처리
# df_masked = mask_dataframe(df.head(20))

# 전체 처리
df_masked = mask_dataframe(df)

마스킹 진행:   2%|▏         | 10/486 [01:04<41:48,  5.27s/it] 

[10/486] 처리 완료


마스킹 진행:   4%|▍         | 20/486 [02:02<1:02:56,  8.10s/it]

[20/486] 처리 완료


마스킹 진행:   6%|▌         | 30/486 [03:47<1:17:24, 10.19s/it]

...

마스킹 진행:  97%|█████████▋| 470/486 [54:42<02:33,  9.60s/it]

[470/486] 처리 완료


마스킹 진행:  99%|█████████▉| 480/486 [56:07<00:50,  8.49s/it]

[480/486] 처리 완료


마스킹 진행: 100%|██████████| 486/486 [56:45<00:00,  7.01s/it]


## 7. 결과 확인 및 저장

In [25]:
def compare_masking(original_df: pd.DataFrame, masked_df: pd.DataFrame, n: int = 3):
    """마스킹 전후 비교 출력"""
    for i in range(min(n, len(original_df))):
        print(f"{'='*60}")
        print(f"[{i+1}] 원본:")
        print(original_df.iloc[i]["content"][:500])
        print(f"\n[{i+1}] 마스킹:")
        print(masked_df.iloc[i]["content"][:500])
    print("=" * 60)

compare_masking(df, df_masked, n=3)

[1] 원본:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질병, 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.

[1] 마스킹:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 [CONDITION], 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.
[2] 원본:
조미진님, 안녕하세요. 2023년 11월 17일 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. 2017년, 2019년, 2021년 모두 ‘정상 B’ 또는 ‘정상 A’ 판정을 받으셨고, 최근 검진에서도 ‘정상 B’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.

현재 가장 집중적으로 관리해야 할 부분은 단백뇨입니다. 단백뇨는 소변으로 단백질이 배출되는 현상인데, 신장이 제대로 기능을 하고 있는지 확인하는 중요한 지표입니다. 신장은 우리 몸의 노폐물을 걸러내는 역할을 하기 때문에, 단백뇨가 지속되면 신장 기능 저하로 이어질 수 있습니다.

30대 여성의 경우, 단백뇨가 심해지면 혈압 상승, 부종, 피로감 등의 증상이 나타날 수 있으며, 심한 경

[2] 마스킹:
[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. 2017년, 2019년, 2021년 모두 ‘[RESULT]’ 또는 ‘[RESULT]’ 판정을 받으셨고, 최근 검진에서도 ‘[RESULT]’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.



In [ ]:
output_path = "MESSAGE_INFO_MASKED.csv"
df_masked.to_csv(output_path, index=False)

print(f"마스킹된 CSV 저장 완료: {output_path}")
print(f"총 {len(df_masked)}개 메시지 처리")

마스킹된 CSV 저장 완료: after_message.csv
총 486개 메시지 처리


## 8. 마스킹 품질 검증

In [27]:
def validate_masking(df_masked: pd.DataFrame) -> dict:
    """마스킹 품질 검증"""
    placeholders = [
        "[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
        "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"
    ]
    
    all_content = " ".join(df_masked["content"].astype(str))
    
    stats = {ph: all_content.count(ph) for ph in placeholders}
    
    # 이메일 유출 검사
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    leaked_emails = re.findall(email_pattern, all_content)
    
    return {"placeholder_counts": stats, "leaked_emails": leaked_emails}


result = validate_masking(df_masked)

print("=== 마스킹 검증 결과 ===")
print(f"총 메시지 수: {len(df_masked)}")
print("\n플레이스홀더 사용 통계:")
for ph, count in result["placeholder_counts"].items():
    if count > 0:
        print(f"  {ph}: {count}회")

if result["leaked_emails"]:
    print(f"\n[경고] 이메일 유출: {result['leaked_emails']}")
else:
    print("\n[OK] 이메일 유출 없음")

=== 마스킹 검증 결과 ===
총 메시지 수: 486

플레이스홀더 사용 통계:
  [NAME]: 203회
  [EMAIL]: 10회
  [PHONE]: 10회
  [ADDRESS]: 10회
  [SSN]: 1회
  [CONDITION]: 97회
  [HEALTH_VALUE]: 170회
  [RESULT]: 61회
  [DATE]: 137회
  [MEDICATION]: 11회

[OK] 이메일 유출 없음


## 9. 요약

### 처리 완료
- 원본 CSV -> 마스킹된 CSV 변환
- Gemma3:27b 로컬 모델 사용 (langchain_ollama)

### 다음 단계
- 마스킹된 데이터로 요약 생성
- 외부 API (Judge)에 전달하여 평가